# Daten laden
Dieses Jupyter Notebook lädt alle benötigten Datensätze nach /data/source

In [ ]:
# Module installieren
!pip install beautifulsoup4

# Importe definieren
import numpy as np
import pandas as pd
import os
import google.colab
from google.colab import files, drive
import requests
import random
import bs4
from bs4 import BeautifulSoup
import warnings
import csv
import zipfile
from zipfile import ZipFile

print(f'requests {requests.__version__}')
print(f'numpy {np.__version__}')
print(f'panda {pd.__version__}')
print(f'google.colab {google.colab.__version__}')
print(f'bs4 {bs4.__version__}')
print(f'csv {csv.__version__}')
print(f'zipfile {zipfile.DEFAULT_VERSION}')

# requests 2.23.0
# numpy 1.21.6
# panda 1.3.5
# google.colab 0.0.1a2
# bs4 4.6.3
# csv 1.0
# zipfile 20

requests 2.23.0
numpy 1.21.6
panda 1.3.5
google.colab 0.0.1a2
bs4 4.6.3
csv 1.0
zipfile 20


## Option Google Drive 

Option um die Speicherung der Daten auf Google Drive zu deaktivieren
Standart => Google Drive aktiv (google_drive = True)

In [ ]:
# Google Drive für Dateizugriff mounten
google_drive = True

if google_drive:
  drive.mount('/content/drive', force_remount=True)
  gdriveroot = '/content/drive/MyDrive/'
else:
  # Gewünschten Pfad setzen wenn Google Drive NICHT verwendet wird
  gdriveroot = ''

!ls {gdriveroot}

meinpfad = gdriveroot + 'ScriptingProjekt/data/'

Mounted at /content/drive
'Colab Notebooks'   hev_hgt.csv   ScriptingProjekt


# Datensatz zu den Wetterstationen beziehen

In [ ]:
# Datensatz von MeteoSchweiz mit Details zu den Wetterstationen herunterladen
url = 'https://data.geo.admin.ch/ch.meteoschweiz.messnetz-automatisch/ch.meteoschweiz.messnetz-automatisch_de.csv'
r = requests.get(url, allow_redirects=True)
dateiname = 'source/ch.meteoschweiz.messnetz-automatisch_de.csv'

open(meinpfad + dateiname, 'wb').write(r.content)

# Ausgabe Dateipfad: /data/source/ch.meteoschweiz.messnetz-automatisch_de.csv

86237

# Datensatz Heizgradtage (HGT) ab Webseite extrahieren

Extrahieren der langjährigen Heizgradtage (HGT) von der Webseite des Hauseigentümerverbandes und diese in einer CSV-Datei speichern.

In [ ]:
# Import der Bibliothek für das Error handling
warnings.filterwarnings('ignore')

# "Parsen" der Homepage für die Extraktion der Heizgradtage
response = requests.get(url="https://www.hev-schweiz.ch/vermieten/nebenkostenabrechnungen/heizgradtage-hgt/", verify = False)
soup = BeautifulSoup(response.content, 'html.parser')

# DEBUG: Anzeigen des Titels der Hompage zur Kontrolle
print(soup.head.title)

# DEBUG: Anzeigen der Extraktion
# print(soup)

# Identifikation der Klasse für die Extraktion
table = soup.find('table', attrs={"class": "contenttable contenttable-0"})

# Anzeigen der exrahierten Werte der Klasse
print(table)

# Werte in Liste übernehmen
table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

# Ausgabe der Liste
print(rows)

# Speichern der Werte in eine CSV Datei als Rohdaten
# dateiname = 'source/Ref_HGT_rawData_20220425.csv'
dateiname = 'source/HGT_raw.csv'
with open(meinpfad + dateiname, 'w', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(row for row in rows if row)

# Ausgabe Dateipfad: /data/source/HGT_raw.csv

<title>Heizgradtage (HGT) - HEV Schweiz</title>
<table class="contenttable contenttable-0"><tr><td>Station</td><td>1981 – 2010 (gem. MeteoSchweiz)</td><td>2001 – 2010 (gem. SIA Norm 381/3)</td><td>2011 – 2020 (gem. SIA Norm 381/3)</td></tr><tr><td>Adelboden</td><td>-</td><td>-</td><td>4338</td></tr><tr><td>Aigle</td><td>-</td><td>-</td><td>2900</td></tr><tr><td>Altdorf</td><td>-</td><td>3073</td><td>2951</td></tr><tr><td>Basel-Binningen</td><td>3035</td><td>2932</td><td>2745</td></tr><tr><td>Bern / Zollikofen</td><td>3526</td><td>3441</td><td>3300</td></tr><tr><td>Buchs-Aarau</td><td>-</td><td>3206</td><td>3047</td></tr><tr><td>La Chaux-de-Fonds</td><td>-</td><td>4315</td><td>4070</td></tr><tr><td>Chur</td><td>-</td><td>3147</td><td>2920</td></tr><tr><td>Cimetta</td><td>-</td><td>-</td><td>4517</td></tr><tr><td>Davos</td><td>-</td><td>5602</td><td>5319</td></tr><tr><td>Disentis / Sedrun</td><td>-</td><td>4331</td><td>4072</td></tr><tr><td>La Dôle</td><td>-</td><td>-</td><td>5260</td></

# Datensatz Normtemperaturen von MeteoSchweiz beziehen

In [ ]:
# Datensatz von MeteoSchweiz herunterladen (ZIP)
url = 'https://www.meteoschweiz.admin.ch/content/dam/meteoswiss/de/Ungebundene-Seiten/Produkte/doc/tnorm9120.zip'
r = requests.get(url, allow_redirects=True)
dateiname = 'source/normtemp.zip'
open(meinpfad + dateiname, 'wb').write(r.content)
print(meinpfad + dateiname)

# ZIP Datei extrahieren

#Erstelle Subfolder für Gemeindedaten
normtemp_pfad = meinpfad + 'source/tnorm9120/'
try:
  os.makedirs(normtemp_pfad)
except FileExistsError:
    pass

with ZipFile(meinpfad + dateiname, 'r') as zipObject:
  listOfFileNames = zipObject.namelist()
  for fileName in listOfFileNames:
    zipObject.extract(fileName, normtemp_pfad)

# Ausgabe Dateipfad: /data/source/tnorm9210/tnorm9120/

/content/drive/MyDrive/ScriptingProjekt/data/source/normtemp.zip


# Datensatz mit den Gemeindekoordinaten beziehen

In [ ]:
# Datensatz von bfs admin herunterladen (ZIP)
url = 'https://dam-api.bfs.admin.ch/hub/api/dam/assets/21224783/master'
r = requests.get(url, allow_redirects=True)
dateiname = 'source/gmd.zip'
open(meinpfad + dateiname, 'wb').write(r.content)
print(meinpfad + dateiname)

# ZIP Datei extrahieren

#Erstelle Subfolder für Gemeindedaten
gmd_pfad = meinpfad + 'source/gmd/'
try:
  os.makedirs(gmd_pfad)
except FileExistsError:
    pass

with ZipFile(meinpfad + dateiname, 'r') as zipObject:
  listOfFileNames = zipObject.namelist()
  for fileName in listOfFileNames:
    # Nur die benoetigte Datei extrahieren
    if 'g1g22.csv' in fileName:
      zipObject.extract(fileName, gmd_pfad)
    elif 'g1g22_li.csv' in fileName:
      zipObject.extract(fileName, gmd_pfad)

# Ausgabe Dateipfad: /data/source/gmd/txt/g1g22.csv
# Ausgabe Dateipfad: /data/source/gmd/txt/g1g22_li.csv

/content/drive/MyDrive/ScriptingProjekt/data/source/gmd.zip
